In [1]:
print ('nvjfndlg')

nvjfndlg


In [1]:
import random
from enum import Enum

# PARTIE 1 — Les Décorateurs

def log_notification(func):
    def wrapper(*args, **kwargs):
        print(f"[logs] Début : {func.__name__}")
        result = func(*args, **kwargs)
        print(f"[log] Fin : {func.__name__}\n")
        return result
    return wrapper


def retry_on_failure(retries=3):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for attempt in range(1, retries + 1):
                try:
                    print(f"Tentative {attempt}/{retries} pour {func.__name__}")
                    return func(*args, **kwargs)
                except Exception as e:
                    print(f"[ERREUR] {e}")
                    if attempt == retries:
                        print(f"Échec après {retries} essais.\n")
        return wrapper
    return decorator


# PARTIE 2 — Types d'urgences et système de priorité

class EmergencyType(Enum):
    WEATHER = "météo"
    SECURITY = "sécurité"
    HEALTH = "santé"
    INFRASTRUCTURE = "infrastructure"
    ACADEMIC = "académique"


class Priority(Enum):
    CRITICAL = 4
    HIGH = 3
    MEDIUM = 2
    LOW = 1


def get_priority(emergency_type):
    if emergency_type in [EmergencyType.SECURITY, EmergencyType.HEALTH]:
        return Priority.CRITICAL
    elif emergency_type == EmergencyType.WEATHER:
        return Priority.HIGH
    elif emergency_type == EmergencyType.INFRASTRUCTURE:
        return Priority.MEDIUM
    else:
        return Priority.LOW


# PARTIE 3 — Les différents canaux d'envoi

class SMSMixin:
    @log_notification
    @retry_on_failure()
    def send_sms(self, message, number):
        if random.random() < 0.2:
            raise Exception("Échec d'envoi SMS")
        print(f" SMS envoyé à {number}: {message}")


class EmailMixin:
    @log_notification
    @retry_on_failure()
    def send_email(self, message, email):
        if random.random() < 0.2:
            raise Exception("Échec d'envoi Email")
        print(f" Email envoyé à {email}: {message}")


class PushNotificationMixin:
    @log_notification
    @retry_on_failure()
    def send_push(self, message, user_id):
        if random.random() < 0.2:
            raise Exception("Échec d'envoi Notification Mobile")
        print(f" Notification mobile envoyée à l'utilisateur {user_id}: {message}")


# PARTIE 4 — Comportements communs

class FormattingMixin:
    def format_message(self, title, body, emergency_type=None):
        formatted = f" {title} \n{body}\n"
        if emergency_type:
            formatted = f"[{emergency_type.value.upper()}] {formatted}"
        return formatted


class ArchiveMixin:
    def archive_notification(self, message):
        print(f" Notification archivée : {message[:30]}...")


class UserPreferenceMixin:
    def prefers_email(self, user):
        return user.get("prefere_email", True)


# PARTIE 5 — Le système de notification académique

class NotificationSystem:
    def __init__(self):
        print(" Système de notification initialisé.")


class AcademicNotifier(NotificationSystem, SMSMixin, EmailMixin,
                     PushNotificationMixin, FormattingMixin,
                     ArchiveMixin, UserPreferenceMixin):

    def send_all_channels(self, message, user):
        try:
            self.send_email(message, user["email"])
        except:
            pass

        try:
            self.send_push(message, user["id"])
        except:
            pass

        if "phone" in user:
            try:
                self.send_sms(message, user["phone"])
            except:
                pass

    def notify(self, user, title, body, emergency_type=EmergencyType.ACADEMIC):
        priority = get_priority(emergency_type)
        message = self.format_message(title, body, emergency_type)

        print(f"[PRIORITÉ] {priority.name} pour {emergency_type.value}")

        if priority == Priority.CRITICAL:
            self.send_all_channels(message, user)
        else:
            if self.prefers_email(user):
                self.send_email(message, user["email"])
            else:
                self.send_push(message, user["id"])

        self.archive_notification(message)

In [3]:
notifier = AcademicNotifier()

 Système de notification initialisé.


In [4]:

user = {
        "id": "user123",
        "email": "etudiant@universite.edu",
        "phone": "+1234567890",
        "prefere_email": True
    }

print("\n=== EXEMPLES DE NOTIFICATIONS ===\n")

notifier.notify(
    user=user,
    title="Alerte météo",
    body="Tempête prévue demain à 14h. Veuillez prendre vos précautions.",
    emergency_type=EmergencyType.WEATHER
)

notifier.notify(
    user=user,
    title="Alerte de sécurité",
    body="Intrusion détectée dans le bâtiment A. Évacuez immédiatement.",
    emergency_type=EmergencyType.SECURITY
)

notifier.notify(
    user=user,
    title="Alerte santé",
    body="Cas de COVID-19 confirmé dans votre classe. Testez-vous rapidement.",
    emergency_type=EmergencyType.HEALTH
)

notifier.notify(
    user=user,
    title="Problème d'infrastructure",
    body="Le système de climatisation du bâtiment B est hors service. Réparation prévue demain.",
    emergency_type=EmergencyType.INFRASTRUCTURE
)

notifier.notify(
    user=user,
    title="Changement d'horaire",
    body="Le cours de mathématiques de demain est reporté à 15h.",
    emergency_type=EmergencyType.ACADEMIC
)


=== EXEMPLES DE NOTIFICATIONS ===

[PRIORITÉ] HIGH pour météo
[logs] Début : wrapper
Tentative 1/3 pour send_email
 Email envoyé à etudiant@universite.edu: [MÉTÉO]  Alerte météo 
Tempête prévue demain à 14h. Veuillez prendre vos précautions.

[log] Fin : wrapper

 Notification archivée : [MÉTÉO]  Alerte météo 
Tempête...
[PRIORITÉ] CRITICAL pour sécurité
[logs] Début : wrapper
Tentative 1/3 pour send_email
[ERREUR] Échec d'envoi Email
Tentative 2/3 pour send_email
[ERREUR] Échec d'envoi Email
Tentative 3/3 pour send_email
 Email envoyé à etudiant@universite.edu: [SÉCURITÉ]  Alerte de sécurité 
Intrusion détectée dans le bâtiment A. Évacuez immédiatement.

[log] Fin : wrapper

[logs] Début : wrapper
Tentative 1/3 pour send_push
 Notification mobile envoyée à l'utilisateur user123: [SÉCURITÉ]  Alerte de sécurité 
Intrusion détectée dans le bâtiment A. Évacuez immédiatement.

[log] Fin : wrapper

[logs] Début : wrapper
Tentative 1/3 pour send_sms
[ERREUR] Échec d'envoi SMS
Tentative 2/3 